In [253]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
import threading
import time
import math
import scipy as sp
from datetime import datetime

## Group Assignment
### Team Number: 08
### Team Member Names: Richard Meng, Avaansh Nanda, Devesh Kotak
### Team Strategy Chosen: Market Beat

We first start by reading the CSV file of tickers and processing the tickers as a list for easy iteration.

In [254]:
# read in the csv file and set the header to none, otherwise it will treat the first ticker as the header
tickers_csv = pd.read_csv("Tickers_Example.csv", header=None)
# turn the csv dataframe into a list
tickers = list(tickers_csv[0])
tickers

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AGN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CELG',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTN',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

Below are some helper functions to help assist filtering our stock data using data threading. 

In [255]:
#  We will be filtering 40 tickers. Each thread will handle 5 tickers which download from yfinance simultaneously, speeding up the process.
START_DATE = "2024-10-01"
END_DATE = "2025-09-30"
tickers_data = {}
monthly_tickers_data = {}

# helper function to download stock data 
def download_yf(ticker_subset, start, end):
    # looping through each ticker and downloading the data from yfinance, and storing in a dict
    for ticker in ticker_subset:
        tickers_data[ticker] = yf.download(ticker, start, end, auto_adjust=False, progress=False)

# helper function to download MONTHLY stock data
def download_monthly_yf(ticker_subset, start, end):
    # looping through each ticker and downloading the data from yfinance, and storing in a dict
    for ticker in ticker_subset:
        monthly_tickers_data[ticker] = yf.download(ticker, start, end, auto_adjust=False, progress=False, interval="1mo")


# stored in a dictionary where each key pair value is the ticker, DataFrame of data
download_yf(tickers, START_DATE, END_DATE)
tickers

ERROR: 
1 Failed download:
ERROR: ['AGN']: YFTzMissingError('possibly delisted; no timezone found')
ERROR: 
1 Failed download:
ERROR: ['CELG']: YFTzMissingError('possibly delisted; no timezone found')
ERROR: 
1 Failed download:
ERROR: ['MON']: YFTzMissingError('possibly delisted; no timezone found')
ERROR: 
1 Failed download:
ERROR: ['RTN']: YFTzMissingError('possibly delisted; no timezone found')


['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AGN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CELG',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTN',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

Next, we begin filtering the stocks. We first must check that they are Canadian or American, which can be done by checking their currency. Then we check if the company is actually listed on its respective market. Lastly, we check the daily average volume (excluding months with less than 18 trading days) and exclude stocks with less than 5000 daily average volume.

In [256]:
# function to check if a singular ticker is valid, then add it to an array of results at its respective index
# this is the helper function for the thread
MIN_TRADING_DAYS = 18
MIN_VOLUME = 5000
VALID_CURRENCIES = ['USD','CAD']

def valid_ticker(ticker, results, index):
    try:
        # store the ticker as a yf.Ticker to find info on currency and sector
        yfTicker = yf.Ticker(ticker)
        # store data to get trading volume + loop through the dates to find number of trading days per month
        ticker_data = tickers_data[ticker]
        # use time.sleep so yFinance does not time out and limit our request rate due to threading
        time.sleep(2)
        # store the currency
        currency = yfTicker.fast_info.get('currency')
        # store the sector
        sector = yfTicker.info.get('sector')
        # again use time.sleep after each yFinance call so it does not limit our request rate
        time.sleep(2)
        # create a date range with the start of each month in range
        month_starts = pd.date_range(start=START_DATE, end=END_DATE, freq="MS")
        # create a date range with the end of each month in range
        month_ends = pd.date_range(start=START_DATE, end=END_DATE, freq="ME")
        # initialize current month index to 0 and num trading days equal to 0
        cur_month_idx = 0
        num_trading_days = 0
        # keep track of the tickers total volume and number of total trading days to find avg daily volume
        total_vol = 0
        total_trading_days = 0
        # initialize date index
        date_idx = 0
        dates = ticker_data.index
        # loop through each date in the ticker data
        while date_idx < len(dates) and cur_month_idx < len(month_starts):
            date = dates[date_idx]
           # if the date is less than the end date of current month it is in
            if date <= month_ends[cur_month_idx]:
                # first add one to the number of trading days
                num_trading_days += 1
                total_trading_days += 1
                # if it is greater than 18, now we check the daily average volume
                #    if its less than 5000 don't add this stock to valid stocks.
                # otherwise, if the number of trading days isn't greater than 18
                #    continue on as then we'll exclude it or wait until it goes above 18
                if num_trading_days >= MIN_TRADING_DAYS:
                    ticker_month_data = yf.download(ticker, start=month_starts[cur_month_idx], end=month_ends[cur_month_idx], auto_adjust=False, progress=False)
                    # again use time.sleep to allow our yFinance requests to go through
                    time.sleep(2)
                    # add to total volume
                    total_vol += ticker_month_data["Volume"].sum().iloc[0]
                    # find the current index of the last date, side="right" gives us the index after the last day of the month's index
                    date_idx = np.searchsorted(dates, month_ends[cur_month_idx], side="right")

            # now we have surpassed the month end and will go to the next month       
            else:
                cur_month_idx += 1
                num_trading_days = 0

            date_idx+=1
        # calculate average daily volume
        avg_daily_vol = total_vol / total_trading_days if total_trading_days > 0 else 0
        # if the average daily volume is less than the minimum or the currency is invalid, don't add the ticker (store it as None).
        if avg_daily_vol<MIN_VOLUME or not(currency in VALID_CURRENCIES):
            results[index] = None
            print(f"removed {ticker}")
        # otherwise, if it's valid, then add the key pair ticker, (sector, currency) to valid_tickers for easy access to a ticker's info.
        else:
            results[index] = (ticker, sector, currency)
    # the except will catch delisted stocks and exclude them from valid_tickers (store it as None)
    except Exception as e:
        print(f"{ticker} is delisted")
        results[index] = None

# function for threading to speed up the process
def filter_tickers(tickers):
    # initialize an list threads to store our threads and valid_tickers to get the return value of the valid_ticker function
    threads = []
    valid_tickers = [""] * len(tickers)

    # loop through the list of possible tickers and get the index and actual ticker
    for index, ticker in enumerate(tickers):
        # start a thread using the valid_ticker function and pass in its respective arguments
        t = threading.Thread(target=valid_ticker, args=(ticker, valid_tickers, index))
        # start the thread and append it to the threads list
        t.start()
        threads.append(t)

    # loop through each thread and join them
    for thread in threads:
        thread.join()
    # use a list comprehension to add to valid_tickers if the ticker is not None.
    return [v for v in valid_tickers if v]

# store the valid_tickers in a list
valid_tickers = filter_tickers(tickers)
# initalize a dictionary to store elements as ticker : (sector, currency)
ticker_dict = {}
# add all valid tickers to the dictionary
for ticker_tup in valid_tickers:
    ticker_dict[ticker_tup[0]] = ticker_tup[1::]

# then change valid_tickers to a list of only the tickers
valid_tickers = [t[0] for t in valid_tickers]

ERROR: $AGN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
ERROR: $RTN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
ERROR: $CELG: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
ERROR: $MON: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


AGN is delisted
RTN is delisted
CELG is delisted
MON is delisted


ERROR: HTTP Error 401: {"finance":{"result":null,"error":{"code":"Unauthorized","description":"Invalid Crumb"}}}
ERROR: HTTP Error 401: {"finance":{"result":null,"error":{"code":"Unauthorized","description":"Invalid Crumb"}}}


Here is the list of all tickers that follow the rules and their respective information stored in a dictionary

In [257]:
print(len(ticker_dict.keys()), len(tickers))
ticker_dict

37 41


{'AAPL': ('Technology', 'USD'),
 'ABBV': ('Healthcare', 'USD'),
 'ABT': ('Healthcare', 'USD'),
 'ACN': ('Technology', 'USD'),
 'AIG': ('Financial Services', 'USD'),
 'AMZN': ('Consumer Cyclical', 'USD'),
 'AXP': ('Financial Services', 'USD'),
 'BA': ('Industrials', 'USD'),
 'BAC': ('Financial Services', 'USD'),
 'BB.TO': ('Technology', 'CAD'),
 'BIIB': ('Healthcare', 'USD'),
 'BK': ('Financial Services', 'USD'),
 'BLK': ('Financial Services', 'USD'),
 'BMY': ('Healthcare', 'USD'),
 'C': ('Financial Services', 'USD'),
 'CAT': ('Industrials', 'USD'),
 'CL': (None, 'USD'),
 'KO': ('Consumer Defensive', 'USD'),
 'LLY': ('Healthcare', 'USD'),
 'LMT': ('Industrials', 'USD'),
 'MO': (None, 'USD'),
 'MRK': ('Healthcare', 'USD'),
 'PEP': ('Consumer Defensive', 'USD'),
 'PFE': ('Healthcare', 'USD'),
 'PG': ('Consumer Defensive', 'USD'),
 'PM': ('Consumer Defensive', 'USD'),
 'PYPL': ('Financial Services', 'USD'),
 'QCOM': ('Technology', 'USD'),
 'RY.TO': ('Financial Services', 'CAD'),
 'SHOP.TO'

Since we must choose 10-25 stocks, we will first rank the top 25 stocks using a metric or scoring system we came up with.

Each stock has a score: 
* $\ 0.5*Sharpe \ Ratio + 0.5*Alpha $

We chose this metric because the Sharpe Ratio provides the risk-adjusted return, while the Alpha indicates the return above the market. 

We chose the $\ Sharpe \ Ratio$ as we still would like to be slightly risk-averse and it provides us a strong indicator of how much a stock returns given its risk, over a long-term period. As for $\ Alpha$, this helps us with our goal, of market beat, by giving a stock's specific return above the market.

We will find the Sharpe Ratio of each stock by finding the returns of each stock, the risk-free rate given by t-bills (dependent on currency), and the standard deviation of each stock.

Then we will find the Alpha of each stock by finding the returns of each stock and the returns of its respective market (also dependent on currency)

Then we output the top 25 stocks in order from highest to least score out of the valid stocks.

In [265]:
# the start date is right after the COVID bubble
DATA_START_DATE = "2022-06-01"
DATA_END_DATE = "2025-11-15"
download_monthly_yf(valid_tickers, start=DATA_START_DATE, end=DATA_END_DATE)


# helper function to get the average returns for the index 
def get_index_returns(index_ticker):
    # downloading monthly data from the index 
    index_data = yf.download(index_ticker, start=DATA_START_DATE, end=DATA_END_DATE, auto_adjust=False, progress=False, interval="1mo")
    index_returns = index_data["Close"].pct_change().dropna().mean().item() # .item() gets the acutal value we need from the series
    return index_returns

# function to retrieve risk free rates for respective markets (American/Canadian)
def get_risk_free_rates():
    # ^TNX - 10 year Treasury Yield, mostly risk free and commonly used to calculate risk-free rate 
    # From Bank Of Canada, current treasury bond yields for 1 year are 2.31% (as of November 12, 2025)
    american_bond_data = yf.download("^TNX", period="1d", auto_adjust=False, progress=False)  # get the most recent rate (most recent closing price)
    american_rf_rate = (tnx['Close'].iloc[-1] / 100).item() # divide by 100 to get decimal 
    canadian_rf_rate = 0.0231
    return (american_rf_rate, canadian_rf_rate)

RF_rates = get_risk_free_rates()
USD_RF = RF_rates[0]
CAD_RF = RF_rates[1]

# function to calculate the sharpe ratio of all the stocks 
def get_sharpe_ratios(tickers):
    sharpe_ratios = {} # dictionary of the sharpe ratios
    # looping through each stock and calculating its sharpe ratio
    for ticker in tickers:
        ticker_currency = ticker_dict[ticker][1]
        RF = USD_RF/12 if ticker_currency == "USD" else CAD_RF/12 # divide by 12 since we look at monthly data here
        returns = monthly_tickers_data[ticker]["Close"].pct_change().dropna()
        Rp = returns.mean() # calculating rate of return of the portfolio 
        std_portfolio = returns.std() # calcualting std of the portfolio 
        
        sharpe_ratio = ((Rp - RF) / std_portfolio) * math.sqrt(12) # multiplying by sqrt(12) to annualize sharpe ratio
        sharpe_ratios[ticker] = sharpe_ratio.item() # adding to the dictionary, .item() to convert from series
    
    return sharpe_ratios

# helper function to get the alpha values (returns vs market returns) of all the stocks 
def get_alpha(tickers):
    alpha_values = {} # dictionary to store ("Ticker": alpha_value) for each stock
    tsx_avg_returns = get_index_returns("^GSPTSE")
    sp_avg_returns = get_index_returns("^GSPC")
    index_avg_returns = None # will choose based on stock's currency
    for ticker in tickers:
        returns = monthly_tickers_data[ticker]["Close"].pct_change().dropna().mean()
        index_ticker = None # the index will either be TSX or S&P 500
        ticker_currency = ticker_dict[ticker][1]
        # if the ticker's currency is CAD, then the benchmark will be TSX, if USD, then S&P 500
        if ticker_currency == "CAD":
            index_avg_returns = tsx_avg_returns
        else:
            index_avg_returns = sp_avg_returns

        alpha = (returns - index_avg_returns) * 12 # * 12 to annualize
        
        alpha_values[ticker] = alpha.item() # adding to the dictionary, .item() to convert from series 

    return alpha_values

# getting the alpha values and sharpe ratios and converting to a list so we can sort 
alpha_values = get_alpha(valid_tickers)
sharpe_ratios = get_sharpe_ratios(valid_tickers)


SHARPE_WEIGHTING = 0.5
ALPHA_WEIGHTING = 0.5

# a dictionary of the total scores of each stock (sharpe + alpha) 
total_scores = {ticker: ALPHA_WEIGHTING*alpha_values[ticker] + SHARPE_WEIGHTING*sharpe_ratios[ticker] for ticker in valid_tickers}
sorted_tickers = sorted(total_scores.keys(), key=total_scores.get, reverse=True) # getting a list of the keys (tickers) of total_scores by score
sorted_scores = {ticker: total_scores[ticker] for ticker in sorted_tickers}

# function to get the top 17 stocks based on their score 
def get_top_stocks(n):
    top_stocks = {} # empty dictionary to store the top n stocks
    for i in range(0, n):
        ticker = sorted_tickers[i]
        score = sorted_scores[ticker]
        top_stocks[ticker] = score

    return top_stocks

top_stocks = get_top_stocks(25)
top_stocks


# sorting the dictionaries in ascending order 
#dict(sorted(alpha_values.items()))

# TODO: We need to factor in fees

# function to create a portfolio with the best 17 stocks based on 50% weighting in sharpe ratio, 50% in alpha.
#def create_portfolio():
 #   dict(sorted(alpha_values.items()))

# NEXT THINGS TO DO
# 1. Make a function which creates a portfolio with the best 17 stocks based on 50% weighting in sharpe ratio, 50% in alpha. - DONE

# 2. Factor in the restriction of MAX 40% can be in one industry <- might take most time - NEXT TIME
# 3. Create the optimal stock weightings using SciPi minimize() function (Richard will do this)
# 4. Once 1, 2,3. are working, expand it to a loop that goes through 10-25 stocks, then compares those portfolios to get the one with the highest return 


{'SHOP.TO': 0.7707782549804608,
 'BK': 0.6961485637229792,
 'LLY': 0.6458566106895436,
 'CAT': 0.604731641490989,
 'AXP': 0.595189683100336,
 'C': 0.4493452475991869,
 'AMZN': 0.42932774459203304,
 'AAPL': 0.4094011583560099,
 'RY.TO': 0.4002018068154875,
 'BLK': 0.30126869411987656,
 'BAC': 0.27863351585891644,
 'PM': 0.24037571775260477,
 'AIG': 0.21994123342651012,
 'ABBV': 0.2175816918852283,
 'TD.TO': 0.21340643099921489,
 'BA': 0.16973048582804878,
 'MO': 0.1553322547394593,
 'QCOM': 0.13213586172585484,
 'BB.TO': 0.06843273579550815,
 'ABT': 0.02498357817710682,
 'USB': -0.03837836895116856,
 'KO': -0.04676678365098663,
 'LMT': -0.04927972477017063,
 'TXN': -0.05091794655874775,
 'UNP': -0.09400715060198088}

Below is a sample of how optimal weightings will work

In [259]:
# initialize variables for initial optimization check
initial_weightings = [1/25]*25
# creates a list of all tickers
top_tickers = list(top_stocks.keys())
CAD_INITIAL_INVESTMENT = 1_000_000
stock_shares = {}

# fetches most recent exchange rate (CAD to USD) because we are keeping everything in CAD
EXCHANGE_RATE = yf.download("CADUSD=X", period="1d", auto_adjust=False, progress=False)['Close'].iloc[-1].item()

# calculates the return of a portfolio with fees included
def portfolio_returns(tickers, weights, investment):
    pf = pd.DataFrame()
    # loop through all tickers and keep track of their index
    for index, ticker in enumerate(tickers):
        ticker_currency = ticker_dict[ticker][1] # get the currency of the current ticker
        # ignore currencies at first and simply allocate the required amount in CAD
        allocation = investment*weights[index] # this is in CAD
        # calculate the number of shares by converting its close price to CAD
        first_close = tickers_data[ticker]["Close"].iloc[0]
        # the number of shares we buy is how many we allocate into the ticker divided by its first closing price (in CAD)
        shares = allocation/first_close.item()
        # dividing by exchange rate to go from USD to CAD (we are keeping our portfolio in CAD)
        fees = min(2.15/EXCHANGE_RATE, (0.001/EXCHANGE_RATE)*shares)
        # amount of money we can allocate to a certain stock subtract its respective fees
        allocation_sub_fees = allocation-fees
        # calculate the amount of shares we can buy of the specific stock
        shares = allocation_sub_fees/first_close
        # put the initial investment in the portfolio
        pf[ticker] = shares*tickers_data[ticker]["Close"]

    # create a column for the total value of the portfolio
    pf["Total"] = pf.sum(axis=1)
    # calculate the total returns of the portfolio
    total_returns = (pf["Total"].iloc[-1] - pf["Total"].iloc[0])/(pf["Total"].iloc[0])
    # change returns from a series to a scalar
    total_returns = total_returns.item()
    # return total portfolio returns as first element and the actual portfolio as the second element
    return float(total_returns), pf

# maximizing a function is the same as minimizing the negative
def negative_portfolio_returns(weights, tickers, investment):
    return -portfolio_returns(tickers, weights, investment)[0]
    
# using scipy to optimize weights
optimal = sp.optimize.minimize(
    fun=negative_portfolio_returns, # fun is the function we are minimizing, so minimizing the negative will maximize the original
    x0=[1/len(top_tickers)] * len(top_tickers), # x0 is the initial guess for which we optimize the function
    args=(top_tickers, CAD_INITIAL_INVESTMENT), # args is the argument of the function we are optimizing
    method='SLSQP', # the algorithm SciPy uses to optimize
    bounds=[(1/(2*len(top_tickers)), 0.15)] * len(top_tickers), # the min and max of each element in the weights
    constraints={'type': 'eq', 'fun': lambda w: np.sum(w) - 1}, # what constraints the weightings must have (sum of weights must equal 1 or 100%)
    options={'ftol': 1e-12} # ftol is just the tolerance that we can take below the constraints (e.g can not be less than 1e-12% of 100%)
)
# the scipy gives optimal weights, minimize function x value
optimal_weights = optimal.x
# scipy also gives the optimal value of the function, which would be the return.
optimal_return = -optimal.fun # we do negative because we minimized the negative function
# returns the optimal portfolio
optimal_portfolio = portfolio_returns(top_tickers, optimal.x, CAD_INITIAL_INVESTMENT)[1]
print(f"Optimal weights: {optimal_weights}")
print(f"Optimal return: {100*optimal_return}%")
display(optimal_portfolio)

Optimal weights: [0.15 0.13 0.02 0.02 0.02 0.15 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
 0.02 0.02 0.02 0.02 0.15 0.02 0.02 0.02 0.02 0.02 0.02]
Optimal return: 53.877087099858045%


,SHOP.TO,BK,LLY,CAT,AXP,C,AMZN,AAPL,RY.TO,BLK,...,MO,QCOM,BB.TO,ABT,USB,KO,LMT,TXN,UNP,Total
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-01,149998.016937,129997.438574,19999.968318,19999.928586,19999.895674,149996.987635,19999.848636,19999.876124,19999.832284,19999.969999,...,19999.449794,19999.830969,149996.987635,19999.753241,19999.369726,19999.609232,19999.953748,19999.861002,19999.885881,9.999845e+05
2024-10-02,151201.056015,129704.933075,20148.982803,20084.536612,20172.642894,150530.864798,19959.875907,20050.270759,19858.582819,20497.388579,...,19865.936816,20306.259621,149085.152703,20013.842858,19904.906176,19804.383104,19912.145771,20181.406022,19820.697579,1.000528e+06
2024-10-03,149998.016937,129467.282817,20024.163453,19933.667284,19999.150349,148783.616139,19657.389381,19952.132418,19695.787848,20374.050728,...,19626.398695,20378.642994,143614.143110,19765.519739,19783.452305,19667.722687,19998.634040,20006.804611,19599.155273,9.892116e+05
2024-10-04,158390.999179,131587.905845,20060.568637,20239.484372,20548.663843,152011.179504,20148.930810,20052.039394,19894.493824,20324.801583,...,19673.520482,20376.229662,147261.482839,19837.726675,20107.329294,19570.109623,19975.856472,20109.979342,19454.989592,1.010721e+06
2024-10-07,155956.608124,132428.838234,20314.730036,20298.609195,20386.342081,151914.107563,19532.072593,19600.249570,19858.582819,20142.364236,...,19602.837053,20139.774734,145437.812975,19964.530542,20003.868970,19246.591736,19989.390034,19967.122954,19292.904956,1.004548e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-23,293641.137680,198643.257352,16890.801684,24019.893955,25425.779021,249954.528121,23843.605090,22494.886100,24543.724158,24233.919587,...,25473.480371,20452.233953,268535.637504,23932.427915,22199.030269,18605.130812,16065.391336,18059.397085,18568.820366,1.486231e+06
2025-09-24,292749.464911,197637.787132,16774.801393,23944.968706,25352.064572,246678.428785,23789.589404,22307.451245,24511.405897,24153.192969,...,25575.576327,20937.210467,270359.307368,23580.195551,22122.560267,18532.617339,16051.526335,18297.491569,18762.670195,1.482458e+06
2025-09-25,283011.922285,199411.063366,16158.396135,23635.583379,25329.725276,247794.728339,23567.043390,22710.613721,24401.280156,24566.675104,...,25599.139467,20470.329336,294978.872274,23478.047842,22055.085322,18401.538155,15988.475488,18059.397085,18896.247064,1.498844e+06


Sample of looping through 10-25 stocks to find best 

In [271]:
# dictionary stores all of the best portfolios from choosing 10-25 stocks
all_optimals = {}
# loop through the numbers 10-25 inclusive to create portfolios with 10-25 stocks inclusive
for i in range(10, 26):
    # get the top n stocks (top 10, top 11, ... , top 25)
    top_n_tickers = top_tickers[:i]
    # optimizes each respective portfolio with 10 stocks, 11 stocks, etc.
    optimal_n = sp.optimize.minimize(
        fun=negative_portfolio_returns,
        x0=[1/i] * i,
        args=(top_n_tickers, CAD_INITIAL_INVESTMENT),
        method='SLSQP',
        bounds=[(1/(2*i), 0.15)] * i,
        constraints={'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        options={'ftol': 1e-12}
    )
    # add all optimal portfolios to the dictionary with its weightings and its return
    all_optimals[i] = (optimal_n.x, -optimal_n.fun)

# initialize max variables
max_return = 0
max_number_stock = 0
# will get the maximum return portfolio and how many stocks we should choose
for n_stocks in list(all_optimals.keys()):
    # the return of the current portfolio we are looking at
    stock_return = all_optimals[n_stocks][1]
    # check if the current portfolio we are looking at has a greater return than the max
    if stock_return > max_return:
        # if it does, then change the max to the current portfolio
        max_return = stock_return
        max_number_stock = n_stocks

# output max return
print(max_return, max_number_stock)
print(all_optimals[max_number_stock][0])
print(top_tickers[:max_number_stock])

# - Market Cap Mix: Must include at least one large-cap (> $10B CAD), and one small-cap (< $2B CAD).
# - No more than 40% of total portfolio value of your portfolio can be in a single sector.

0.5730911723681569 19
[0.15       0.15       0.02631579 0.02631579 0.02631579 0.15
 0.02631579 0.02631579 0.02631579 0.02631579 0.02631579 0.02631579
 0.02631579 0.02631579 0.02631579 0.03157895 0.02631579 0.02631579
 0.15      ]
['SHOP.TO', 'BK', 'LLY', 'CAT', 'AXP', 'C', 'AMZN', 'AAPL', 'RY.TO', 'BLK', 'BAC', 'PM', 'AIG', 'ABBV', 'TD.TO', 'BA', 'MO', 'QCOM', 'BB.TO']


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.
- Richard Meng
- Avaansh Nanda
- Devesh Kotak